In [1]:
import warnings
warnings.filterwarnings("ignore")


In [2]:

!pip install comtypes

In [3]:
import threading
import time
import datetime as dt
import json
import os
import logging
import gc
from pywifi import PyWiFi

In [4]:
FILE_PATH = 'rssi_data.json'

# Configure logging
logging.basicConfig(
    filename=f'rssi_{dt.datetime.now().strftime("%Y%m%d%H%M%S")}.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Threading control
stop_event = threading.Event()  # Used to stop the data collection thread
running = False  # Flag to indicate whether data collection is ongoing
crowd_count_lock = threading.Lock()  # Lock to handle concurrent access to crowd_count
crowd_count = None  # Variable to store the crowd count

def init(file_path):
    """Initialize the JSON file by creating an opening bracket for the array."""
    with open(file_path, 'w') as file:
        file.write('[')  # Start the JSON array
    logging.info(f"Initialized JSON file: {file_path}")

def append_record(file_path, record):
    """Append a single record to the JSON file without loading the entire file."""
    with open(file_path, 'a+') as file:
        file.seek(0, os.SEEK_END)
        # Add a comma before appending a new record (if it's not the first record)
        if file.tell() > 1:
            file.write(',')
        
        json.dump(record, file, indent=2)
        file.write('\n')
    logging.info(f"Appended new record to JSON file: {file_path}")

def finalize(file_path):
    """Add the closing bracket to finalize the JSON array."""
    with open(file_path, 'a') as file:
        file.write(']')  # End the JSON array
    logging.info(f"Finalized and closed JSON file: {file_path}")

def capture_rssi():
    """Capture RSSI data in a separate thread and handle stopping smoothly."""
    wifi = PyWiFi()
    iface = wifi.interfaces()[0]  # Assuming first interface is the one we want

    try:
        next_scan_time = time.time()  # Set initial scan time
        
        while not stop_event.is_set():
            start_time = time.time()

            iface.scan()  # Start a new scan
            results = iface.scan_results()

            # Get the current timestamp (without milliseconds)
            time_string = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            for network in results:
                # Safely access the crowd_count with a lock
                with crowd_count_lock:
                    current_crowd_count = crowd_count

                # Create a record to store in the JSON file
                record = {
                    'timestamp': time_string,  # Unique timestamp (without milliseconds)
                    'ssid': network.ssid,
                    'bssid': network.bssid,
                    'rssi': network.signal,
                    'crowd_count': current_crowd_count  # Include the crowd count
                }
                # Append the record to the JSON file
                append_record(FILE_PATH, record)

            # Force garbage collection to free up memory periodically
            gc.collect()

            # Print status to terminal
            print(f"Data collected at: {time_string} with crowd count: {current_crowd_count}")

            # Calculate the time for the next scan
            next_scan_time += 1.0
            sleep_duration = max(0, next_scan_time - time.time())  # Ensure non-negative sleep time
            time.sleep(sleep_duration)

    except Exception as e:
        print(f"Error during Wi-Fi scan: {e}")

# Function to start logging RSSI data
def start_logging():
    global running, capture_thread, crowd_count  # Ensure crowd_count is global
    if not running:
        # Take crowd count input if it's not set or if user wants to update it
        if crowd_count is None:
            crowd_count_input = input("Enter initial crowd count: ")
            with crowd_count_lock:
                crowd_count = crowd_count_input

        running = True
        stop_event.clear()  # Clear the stop event
        print(f"Data collection started with crowd count: {crowd_count}...")

        # Start the RSSI capture in a new thread
        capture_thread = threading.Thread(target=capture_rssi)
        capture_thread.start()

# Function to stop logging RSSI data
def stop_logging():
    global running
    if running:
        running = False
        stop_event.set()  # Signal the thread to stop
        print("Stopping data collection...")

        # Wait for the thread to finish
        capture_thread.join()

        # Finalize the JSON file
        finalize(FILE_PATH)
        print("Data collection stopped. Ready for next command.")

def main():
    global crowd_count
    # Initialize the JSON file
    init(FILE_PATH)

    while True:
        command = input("Enter 'Y' to start logging, 'N' to stop logging, 'X' to quit, or provide new crowd count: ").upper()

        if command == 'Y':
            start_logging()
        elif command == 'N':
            if running:
                stop_logging()
            else:
                print("Data collection is not running.")
        elif command == 'X':
            if running:
                stop_logging()
            print("Exiting the program.")
            break
        else:
            # If the command is not 'Y', 'N', or 'X', assume it's a new crowd count
            try:
                new_crowd_count = int(command)  # Set new crowd count
                with crowd_count_lock:
                    crowd_count = new_crowd_count  # Update the crowd count
                print(f"Crowd count updated to: {crowd_count}")
            except ValueError:
                print("Invalid input. Please enter 'Y', 'N', 'X', or a valid crowd count.")

if __name__ == '__main__':
    main()

Enter 'Y' to start logging, 'N' to stop logging, 'X' to quit, or provide new crowd count: Y
Enter initial crowd count: 6
Data collection started with crowd count: 6...
Data collected at: 2024-10-14 18:47:13 with crowd count: 6
Data collected at: 2024-10-14 18:47:14 with crowd count: 6
Data collected at: 2024-10-14 18:47:15 with crowd count: 6
Data collected at: 2024-10-14 18:47:16 with crowd count: 6
Data collected at: 2024-10-14 18:47:17 with crowd count: 6
Data collected at: 2024-10-14 18:47:18 with crowd count: 6
Data collected at: 2024-10-14 18:47:19 with crowd count: 6
Data collected at: 2024-10-14 18:47:20 with crowd count: 6
Data collected at: 2024-10-14 18:47:21 with crowd count: 6
Data collected at: 2024-10-14 18:47:22 with crowd count: 6
Data collected at: 2024-10-14 18:47:23 with crowd count: 6
Enter 'Y' to start logging, 'N' to stop logging, 'X' to quit, or provide new crowd count: N
Data collected at: 2024-10-14 18:47:24 with crowd count: 6
Stopping data collection...
Data

Data collected at: 2024-10-14 18:49:25 with crowd count: 10
Data collected at: 2024-10-14 18:49:26 with crowd count: 10
Data collected at: 2024-10-14 18:49:27 with crowd count: 10
Data collected at: 2024-10-14 18:49:28 with crowd count: 10
Data collected at: 2024-10-14 18:49:29 with crowd count: 10
Data collected at: 2024-10-14 18:49:30 with crowd count: 10
Data collected at: 2024-10-14 18:49:31 with crowd count: 10
Data collected at: 2024-10-14 18:49:32 with crowd count: 10
Data collected at: 2024-10-14 18:49:33 with crowd count: 10
Data collected at: 2024-10-14 18:49:34 with crowd count: 10
Data collected at: 2024-10-14 18:49:35 with crowd count: 10
Data collected at: 2024-10-14 18:49:36 with crowd count: 10
Data collected at: 2024-10-14 18:49:37 with crowd count: 10
Data collected at: 2024-10-14 18:49:38 with crowd count: 10
Data collected at: 2024-10-14 18:49:39 with crowd count: 10
Data collected at: 2024-10-14 18:49:40 with crowd count: 10
Data collected at: 2024-10-14 18:49:41 w

Data collected at: 2024-10-14 18:51:36 with crowd count: 11
Data collected at: 2024-10-14 18:51:37 with crowd count: 11
Data collected at: 2024-10-14 18:51:38 with crowd count: 11
Data collected at: 2024-10-14 18:51:39 with crowd count: 11
Data collected at: 2024-10-14 18:51:40 with crowd count: 11
Data collected at: 2024-10-14 18:51:41 with crowd count: 11
Data collected at: 2024-10-14 18:51:42 with crowd count: 11
Data collected at: 2024-10-14 18:51:43 with crowd count: 11
Data collected at: 2024-10-14 18:51:44 with crowd count: 11
Data collected at: 2024-10-14 18:51:45 with crowd count: 11
Data collected at: 2024-10-14 18:51:46 with crowd count: 11
Data collected at: 2024-10-14 18:51:47 with crowd count: 11
Data collected at: 2024-10-14 18:51:48 with crowd count: 11
Data collected at: 2024-10-14 18:51:49 with crowd count: 11
Data collected at: 2024-10-14 18:51:50 with crowd count: 11
Data collected at: 2024-10-14 18:51:51 with crowd count: 11
Data collected at: 2024-10-14 18:51:52 w

Data collected at: 2024-10-14 18:53:49 with crowd count: 11
Data collected at: 2024-10-14 18:53:50 with crowd count: 11
Data collected at: 2024-10-14 18:53:51 with crowd count: 11
Data collected at: 2024-10-14 18:53:52 with crowd count: 11
Data collected at: 2024-10-14 18:53:53 with crowd count: 11
Data collected at: 2024-10-14 18:53:54 with crowd count: 11
Data collected at: 2024-10-14 18:53:55 with crowd count: 11
Data collected at: 2024-10-14 18:53:56 with crowd count: 11
Data collected at: 2024-10-14 18:53:57 with crowd count: 11
Data collected at: 2024-10-14 18:53:58 with crowd count: 11
Data collected at: 2024-10-14 18:53:59 with crowd count: 11
Data collected at: 2024-10-14 18:54:00 with crowd count: 11
Data collected at: 2024-10-14 18:54:01 with crowd count: 11
Data collected at: 2024-10-14 18:54:02 with crowd count: 11
Data collected at: 2024-10-14 18:54:03 with crowd count: 11
Data collected at: 2024-10-14 18:54:04 with crowd count: 11
Data collected at: 2024-10-14 18:54:05 w

Data collected at: 2024-10-14 18:56:00 with crowd count: 9
Data collected at: 2024-10-14 18:56:01 with crowd count: 9
Data collected at: 2024-10-14 18:56:02 with crowd count: 9
Data collected at: 2024-10-14 18:56:03 with crowd count: 9
Data collected at: 2024-10-14 18:56:04 with crowd count: 9
Data collected at: 2024-10-14 18:56:05 with crowd count: 9
Data collected at: 2024-10-14 18:56:06 with crowd count: 9
Data collected at: 2024-10-14 18:56:07 with crowd count: 9
Data collected at: 2024-10-14 18:56:08 with crowd count: 9
Data collected at: 2024-10-14 18:56:09 with crowd count: 9
Data collected at: 2024-10-14 18:56:10 with crowd count: 9
Data collected at: 2024-10-14 18:56:11 with crowd count: 9
Data collected at: 2024-10-14 18:56:12 with crowd count: 9
Data collected at: 2024-10-14 18:56:13 with crowd count: 9
Data collected at: 2024-10-14 18:56:14 with crowd count: 9
Data collected at: 2024-10-14 18:56:15 with crowd count: 9
Data collected at: 2024-10-14 18:56:16 with crowd count:

...

Data collected at: 2024-10-14 19:49:59 with crowd count: 12
Data collected at: 2024-10-14 19:50:00 with crowd count: 12
Data collected at: 2024-10-14 19:50:01 with crowd count: 12
Data collected at: 2024-10-14 19:50:02 with crowd count: 12
Data collected at: 2024-10-14 19:50:03 with crowd count: 12
Data collected at: 2024-10-14 19:50:04 with crowd count: 12
Data collected at: 2024-10-14 19:50:05 with crowd count: 12
Data collected at: 2024-10-14 19:50:06 with crowd count: 12
Data collected at: 2024-10-14 19:50:06 with crowd count: 12
Data collected at: 2024-10-14 19:50:07 with crowd count: 12
Data collected at: 2024-10-14 19:50:08 with crowd count: 12
Data collected at: 2024-10-14 19:50:09 with crowd count: 12
Data collected at: 2024-10-14 19:50:10 with crowd count: 12
Data collected at: 2024-10-14 19:50:11 with crowd count: 12
Data collected at: 2024-10-14 19:50:12 with crowd count: 12
Data collected at: 2024-10-14 19:50:13 with crowd count: 12
Data collected at: 2024-10-14 19:50:14 w

Data collected at: 2024-10-14 19:52:16 with crowd count: 13
Data collected at: 2024-10-14 19:52:17 with crowd count: 13
Data collected at: 2024-10-14 19:52:18 with crowd count: 13
Data collected at: 2024-10-14 19:52:19 with crowd count: 13
Data collected at: 2024-10-14 19:52:20 with crowd count: 13
Data collected at: 2024-10-14 19:52:22 with crowd count: 13
Data collected at: 2024-10-14 19:52:23 with crowd count: 13
Data collected at: 2024-10-14 19:52:24 with crowd count: 13
Data collected at: 2024-10-14 19:52:25 with crowd count: 13
Data collected at: 2024-10-14 19:52:26 with crowd count: 13
Data collected at: 2024-10-14 19:52:27 with crowd count: 13
Data collected at: 2024-10-14 19:52:27 with crowd count: 13
Data collected at: 2024-10-14 19:52:28 with crowd count: 13
Data collected at: 2024-10-14 19:52:29 with crowd count: 13
Data collected at: 2024-10-14 19:52:30 with crowd count: 13
Data collected at: 2024-10-14 19:52:31 with crowd count: 13
Data collected at: 2024-10-14 19:52:32 w

Data collected at: 2024-10-14 19:54:28 with crowd count: 12
Data collected at: 2024-10-14 19:54:29 with crowd count: 12
Data collected at: 2024-10-14 19:54:30 with crowd count: 12
Data collected at: 2024-10-14 19:54:31 with crowd count: 12
Data collected at: 2024-10-14 19:54:32 with crowd count: 12
Data collected at: 2024-10-14 19:54:33 with crowd count: 12
Data collected at: 2024-10-14 19:54:34 with crowd count: 12
Data collected at: 2024-10-14 19:54:35 with crowd count: 12
Data collected at: 2024-10-14 19:54:36 with crowd count: 12
Data collected at: 2024-10-14 19:54:37 with crowd count: 12
Data collected at: 2024-10-14 19:54:38 with crowd count: 12
Data collected at: 2024-10-14 19:54:39 with crowd count: 12
Data collected at: 2024-10-14 19:54:40 with crowd count: 12
Data collected at: 2024-10-14 19:54:41 with crowd count: 12
Data collected at: 2024-10-14 19:54:42 with crowd count: 12
Data collected at: 2024-10-14 19:54:43 with crowd count: 12
Data collected at: 2024-10-14 19:54:44 w

Data collected at: 2024-10-14 19:56:45 with crowd count: 12
Data collected at: 2024-10-14 19:56:46 with crowd count: 12
Data collected at: 2024-10-14 19:56:47 with crowd count: 12
Data collected at: 2024-10-14 19:56:48 with crowd count: 12
Data collected at: 2024-10-14 19:56:49 with crowd count: 12
Data collected at: 2024-10-14 19:56:50 with crowd count: 12
Data collected at: 2024-10-14 19:56:51 with crowd count: 12
Data collected at: 2024-10-14 19:56:52 with crowd count: 12
Data collected at: 2024-10-14 19:56:53 with crowd count: 12
Data collected at: 2024-10-14 19:56:55 with crowd count: 12
Data collected at: 2024-10-14 19:56:55 with crowd count: 12
Data collected at: 2024-10-14 19:56:56 with crowd count: 12
Data collected at: 2024-10-14 19:56:57 with crowd count: 12
Data collected at: 2024-10-14 19:56:58 with crowd count: 12
Data collected at: 2024-10-14 19:56:59 with crowd count: 12
Data collected at: 2024-10-14 19:57:00 with crowd count: 12
Data collected at: 2024-10-14 19:57:01 w

Data collected at: 2024-10-14 19:59:02 with crowd count: 9
Data collected at: 2024-10-14 19:59:03 with crowd count: 9
Data collected at: 2024-10-14 19:59:04 with crowd count: 9
Data collected at: 2024-10-14 19:59:05 with crowd count: 9
Data collected at: 2024-10-14 19:59:06 with crowd count: 9
Data collected at: 2024-10-14 19:59:07 with crowd count: 9
Data collected at: 2024-10-14 19:59:08 with crowd count: 9
Data collected at: 2024-10-14 19:59:09 with crowd count: 9
Data collected at: 2024-10-14 19:59:10 with crowd count: 9
Data collected at: 2024-10-14 19:59:11 with crowd count: 9
Data collected at: 2024-10-14 19:59:12 with crowd count: 9
Data collected at: 2024-10-14 19:59:13 with crowd count: 9
Data collected at: 2024-10-14 19:59:14 with crowd count: 9
Data collected at: 2024-10-14 19:59:15 with crowd count: 9
Data collected at: 2024-10-14 19:59:16 with crowd count: 9
Data collected at: 2024-10-14 19:59:17 with crowd count: 9
Data collected at: 2024-10-14 19:59:18 with crowd count: